this notebook is based on my conversation with chatGPT from May 21, 2024: https://chatgpt.com/share/ddf67ef8-6574-41b1-9a04-08f8f626174a 

### calculate fair allocation of clinic hours in a mock set up of 10 authorities:

In [6]:
import pandas as pd

# Sample data
data = {
    'Authority': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10'],
    'Population': [50000, 70000, 30000, 40000, 60000, 50000, 80000, 45000, 55000, 60000],
    'Clinics': [5, 7, 3, 4, 6, 5, 8, 4, 5, 6],
    'Working Hours': [2000, 2500, 1500, 1800, 2300, 2000, 2600, 1800, 2200, 2400]
}

df = pd.DataFrame(data)

# Calculate total population and total working hours
total_population = df['Population'].sum()
total_working_hours = df['Working Hours'].sum()

# Calculate population proportion and allocate working hours
df['Population Proportion'] = df['Population'] / total_population
df['Allocated Working Hours'] = df['Population Proportion'] * total_working_hours

data_df = pd.DataFrame(data)
data_df['Allocated Working Hours'] = df['Allocated Working Hours']

# Display the results
# import ace_tools as tools; tools.display_dataframe_to_user(name="Allocated Working Hours Distribution", dataframe=df)

print(df[['Authority', 'Population', 'Allocated Working Hours']])


  Authority  Population  Allocated Working Hours
0        A1       50000              1953.703704
1        A2       70000              2735.185185
2        A3       30000              1172.222222
3        A4       40000              1562.962963
4        A5       60000              2344.444444
5        A6       50000              1953.703704
6        A7       80000              3125.925926
7        A8       45000              1758.333333
8        A9       55000              2149.074074
9       A10       60000              2344.444444


In [7]:
data_df

,Authority,Population,Clinics,Working Hours,Allocated Working Hours
0,A1,50000,5,2000,1953.703704
1,A2,70000,7,2500,2735.185185
2,A3,30000,3,1500,1172.222222
3,A4,40000,4,1800,1562.962963
4,A5,60000,6,2300,2344.444444
5,A6,50000,5,2000,1953.703704
6,A7,80000,8,2600,3125.925926
7,A8,45000,4,1800,1758.333333
8,A9,55000,5,2200,2149.074074
9,A10,60000,6,2400,2344.444444


### calculate fair allocation of doctors in a mock set up of 10 authorities:
there is a bug in this code - the final allocation is identical to the initial allocation

In [8]:
import pandas as pd
import numpy as np

# Sample data
data = {
    'Authority': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10'],
    'Population': [50000, 70000, 30000, 40000, 60000, 50000, 80000, 45000, 55000, 60000],
    'Doctors': [20, 28, 12, 16, 24, 20, 32, 18, 22, 24]
}

df = pd.DataFrame(data)

# Calculate total population and total number of doctors
total_population = df['Population'].sum()
total_doctors = df['Doctors'].sum()

# Calculate ideal number of doctors
df['Ideal Doctors'] = (df['Population'] / total_population) * total_doctors

# Initial allocation (floor of ideal doctors)
df['Initial Doctors'] = np.floor(df['Ideal Doctors']).astype(int)

# Calculate remaining doctors to be assigned
remaining_doctors = total_doctors - df['Initial Doctors'].sum()

# Calculate remainders
df['Remainder'] = df['Ideal Doctors'] - df['Initial Doctors']

# Distribute remaining doctors based on largest remainders
df = df.sort_values(by='Remainder', ascending=False)
df.iloc[:remaining_doctors, df.columns.get_loc('Initial Doctors')] += 1

# Final allocation
df = df.sort_values(by='Authority')
df['Final Doctors'] = df['Initial Doctors']

# Create summary dataframe
summary_df = df[['Authority', 'Doctors', 'Initial Doctors', 'Final Doctors']].copy()
summary_df.columns = ['Authority', 'Current Doctors', 'Initial Allocation', 'Final Allocation']



print(summary_df)


  Authority  Current Doctors  Initial Allocation  Final Allocation
0        A1               20                  20                20
9       A10               24                  24                24
1        A2               28                  28                28
2        A3               12                  12                12
3        A4               16                  16                16
4        A5               24                  24                24
5        A6               20                  20                20
6        A7               32                  32                32
7        A8               18                  18                18
8        A9               22                  22                22


In [12]:
summary_df["Current Doctors per 1000"].var()

0.00015698223733938012

In [13]:
summary_df["Final Doctors per 1000"].var()

6.160635550516481e-05

### calculate fair allocation of doctors in a mock set up of 10 authorities - take into account the distances between authorities:
(there is a bug in this code as the inital allocation is identical to the final allocation)c

In [16]:
import pandas as pd
import numpy as np

# Sample data with a different initial number of doctors
data = {
    'Authority': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10'],
    'Population': [50000, 70000, 30000, 40000, 60000, 50000, 80000, 45000, 55000, 60000],
    'Current Doctors': [10, 15, 7, 8, 12, 10, 18, 9, 11, 12]
}

# Distance matrix (example distances in km)
distances = pd.DataFrame({
    'A1': [0, 10, 20, 30, 40, 50, 60, 70, 80, 90],
    'A2': [10, 0, 15, 25, 35, 45, 55, 65, 75, 85],
    'A3': [20, 15, 0, 10, 20, 30, 40, 50, 60, 70],
    'A4': [30, 25, 10, 0, 10, 20, 30, 40, 50, 60],
    'A5': [40, 35, 20, 10, 0, 10, 20, 30, 40, 50],
    'A6': [50, 45, 30, 20, 10, 0, 10, 20, 30, 40],
    'A7': [60, 55, 40, 30, 20, 10, 0, 10, 20, 30],
    'A8': [70, 65, 50, 40, 30, 20, 10, 0, 10, 20],
    'A9': [80, 75, 60, 50, 40, 30, 20, 10, 0, 10],
    'A10': [90, 85, 70, 60, 50, 40, 30, 20, 10, 0]
}, index=['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10'])

df = pd.DataFrame(data)

# Calculate total population and total number of doctors
total_population = df['Population'].sum()
total_doctors = df['Current Doctors'].sum()

# Calculate ideal number of doctors
df['Ideal Doctors'] = (df['Population'] / total_population) * total_doctors

# Adjust population based on accessibility
accessibility_factor = 0.1  # Example factor to weight the distance effect

def adjusted_population(row):
    authority = row['Authority']
    neighbors = distances.loc[authority]
    # Invert distances for accessibility (shorter distance = higher accessibility)
    accessibility = (1 / (neighbors + 1))  # +1 to avoid division by zero
    accessibility_score = (df.set_index('Authority').loc[neighbors.index, 'Population'] * accessibility).sum()
    return row['Population'] + accessibility_factor * accessibility_score

df['Adjusted Population'] = df.apply(adjusted_population, axis=1)
adjusted_total_population = df['Adjusted Population'].sum()

# Calculate adjusted ideal number of doctors
df['Adjusted Ideal Doctors'] = (df['Adjusted Population'] / adjusted_total_population) * total_doctors

# Initial allocation (floor of adjusted ideal doctors)
df['Initial Allocation'] = np.floor(df['Adjusted Ideal Doctors']).astype(int)

# Calculate remaining doctors to be assigned
remaining_doctors = total_doctors - df['Initial Allocation'].sum()

# Calculate remainders
df['Remainder'] = df['Adjusted Ideal Doctors'] - df['Initial Allocation']

# Distribute remaining doctors based on largest remainders
df = df.sort_values(by='Remainder', ascending=False)
df.iloc[:remaining_doctors, df.columns.get_loc('Initial Allocation')] += 1

# Final allocation
df = df.sort_values(by='Authority')
df['Final Allocation'] = df['Initial Allocation']

# Calculate doctors per 1000 people
df['Current Doctors per 1000'] = (df['Current Doctors'] / df['Population']) * 1000
df['Final Doctors per 1000'] = (df['Final Allocation'] / df['Population']) * 1000

# Create summary dataframe
summary_df = df[['Authority', 'Population', 'Current Doctors', 'Initial Allocation', 'Final Allocation', 'Current Doctors per 1000', 'Final Doctors per 1000']].copy()

# import ace_tools as tools; tools.display_dataframe_to_user(name="Doctor Allocation Summary with Accessibility", dataframe=summary_df)

summary_df


,Authority,Population,Current Doctors,Initial Allocation,Final Allocation,Current Doctors per 1000,Final Doctors per 1000
0,A1,50000,10,10,10,0.200000,0.200000
9,A10,60000,12,12,12,0.200000,0.200000
1,A2,70000,15,14,14,0.214286,0.200000
2,A3,30000,7,6,6,0.233333,0.200000
3,A4,40000,8,8,8,0.200000,0.200000
4,A5,60000,12,13,13,0.200000,0.216667
5,A6,50000,10,11,11,0.200000,0.220000
6,A7,80000,18,17,17,0.225000,0.212500
7,A8,45000,9,10,10,0.200000,0.222222
8,A9,55000,11,11,11,0.200000,0.200000


### calculate fair allocation of doctors in a mock set up of 10 authorities - take into account the distances between authorities:
(there is a bug in this code as the inital allocation is identical to the final allocation)

In [21]:
import pandas as pd
import numpy as np

# Generate sample data for 100 authorities
np.random.seed(42)
authorities = [f'A{i}' for i in range(1, 101)]
populations = np.random.randint(1000, 100000, size=100)
current_doctors = np.random.randint(1, 50, size=100)

data = {
    'Authority': authorities,
    'Population': populations,
    'Current Doctors': current_doctors
}

# Generate a random distance matrix with significant variations
distances = np.random.randint(1, 1000, size=(100, 100))
np.fill_diagonal(distances, 0)
distances = pd.DataFrame(distances, index=authorities, columns=authorities)

df = pd.DataFrame(data)

# Calculate total population and total number of doctors
total_population = df['Population'].sum()
total_doctors = df['Current Doctors'].sum()

# Calculate ideal number of doctors
df['Ideal Doctors'] = (df['Population'] / total_population) * total_doctors

# Adjust population based on accessibility
accessibility_factor = 0.1  # Example factor to weight the distance effect

def adjusted_population(row):
    authority = row['Authority']
    neighbors = distances.loc[authority]
    # Invert distances for accessibility (shorter distance = higher accessibility)
    accessibility = (1 / (neighbors + 1))  # +1 to avoid division by zero
    accessibility_score = (df.set_index('Authority').loc[neighbors.index, 'Population'] * accessibility).sum()
    return row['Population'] + accessibility_factor * accessibility_score

df['Adjusted Population'] = df.apply(adjusted_population, axis=1)
adjusted_total_population = df['Adjusted Population'].sum()

# Calculate adjusted ideal number of doctors
df['Adjusted Ideal Doctors'] = (df['Adjusted Population'] / adjusted_total_population) * total_doctors

# Initial allocation (floor of adjusted ideal doctors)
df['Initial Allocation'] = np.floor(df['Adjusted Ideal Doctors']).astype(int)

# Calculate remaining doctors to be assigned
remaining_doctors = total_doctors - df['Initial Allocation'].sum()

# Calculate remainders
df['Remainder'] = df['Adjusted Ideal Doctors'] - df['Initial Allocation']

# Distribute remaining doctors based on largest remainders
df = df.sort_values(by='Remainder', ascending=False)
df.iloc[:remaining_doctors, df.columns.get_loc('Initial Allocation')] += 1

# Final allocation
df = df.sort_values(by='Authority')
df['Final Allocation'] = df['Initial Allocation']

# Calculate doctors per 1000 people
df['Current Doctors per 1000'] = (df['Current Doctors'] / df['Population']) * 1000
df['Final Doctors per 1000'] = (df['Final Allocation'] / df['Population']) * 1000

# Create summary dataframe
summary_df = df[['Authority', 'Population', 'Current Doctors', 'Initial Allocation', 'Final Allocation', 'Current Doctors per 1000', 'Final Doctors per 1000']].copy()

# import ace_tools as tools; tools.display_dataframe_to_user(name="Doctor Allocation Summary with Accessibility", dataframe=summary_df)

summary_df


,Authority,Population,Current Doctors,Initial Allocation,Final Allocation,Current Doctors per 1000,Final Doctors per 1000
0,A1,16795,24,9,9,1.428997,0.535874
9,A10,61263,35,31,31,0.571307,0.506015
99,A100,3568,15,2,2,4.204036,0.560538
10,A11,17023,33,10,10,1.938554,0.587441
11,A12,42090,5,21,21,0.118793,0.498931
...,...,...,...,...,...,...,...
94,A95,39044,37,19,19,0.947649,0.486630
95,A96,52214,33,26,26,0.632014,0.497951
96,A97,62228,42,30,30,0.674937,0.482098
97,A98,49984,44,24,24,0.880282,0.480154


In [22]:
summary_df["Current Doctors per 1000"].var()

6.326196949968779

In [23]:

summary_df["Final Doctors per 1000"].var()

0.021226084500394905

In [24]:
(summary_df["Initial Allocation"] - summary_df["Final Allocation"]).max()

0

### calculate fair allocation of doctors in a mock set up of 10 authorities - take into account the distances between authorities:
(the bug is now fixed)

In [28]:
import pandas as pd
import numpy as np

# Generate sample data for 100 authorities
np.random.seed(42)
authorities = [f'A{i}' for i in range(1, 101)]
populations = np.random.randint(1000, 100000, size=100)
current_doctors = np.random.randint(1, 50, size=100)

data = {
    'Authority': authorities,
    'Population': populations,
    'Current Doctors': current_doctors
}

# Generate a random distance matrix with significant variations
distances = np.random.randint(1, 1000, size=(100, 100))
np.fill_diagonal(distances, 0)
distances = pd.DataFrame(distances, index=authorities, columns=authorities)

df = pd.DataFrame(data)

# Calculate total population and total number of doctors
total_population = df['Population'].sum()
total_doctors = df['Current Doctors'].sum()

# Calculate ideal number of doctors
df['Ideal Doctors'] = (df['Population'] / total_population) * total_doctors

# Adjust population based on accessibility
accessibility_factor = 0.1  # Example factor to weight the distance effect

def adjusted_population(row):
    authority = row['Authority']
    neighbors = distances.loc[authority]
    # Invert distances for accessibility (shorter distance = higher accessibility)
    accessibility = (1 / (neighbors + 1))  # +1 to avoid division by zero
    accessibility_score = (df.set_index('Authority').loc[neighbors.index, 'Population'] * accessibility).sum()
    return row['Population'] + accessibility_factor * accessibility_score

df['Adjusted Population'] = df.apply(adjusted_population, axis=1)
adjusted_total_population = df['Adjusted Population'].sum()

# Calculate adjusted ideal number of doctors
df['Adjusted Ideal Doctors'] = (df['Adjusted Population'] / adjusted_total_population) * total_doctors

# Initial allocation (floor of adjusted ideal doctors)
df['Initial Allocation'] = np.floor(df['Adjusted Ideal Doctors']).astype(int)

# Calculate remaining doctors to be assigned
remaining_doctors = total_doctors - df['Initial Allocation'].sum()

# Calculate remainders
df['Remainder'] = df['Adjusted Ideal Doctors'] - df['Initial Allocation']

# Distribute remaining doctors based on largest remainders
df = df.sort_values(by='Remainder', ascending=False)
df['Final Allocation'] = df['Initial Allocation'].copy()

for i in range(remaining_doctors):
    df.iloc[i, df.columns.get_loc('Final Allocation')] += 1

# Final allocation sorted by authority
df = df.sort_values(by='Authority')

# Calculate doctors per 1000 people
df['Current Doctors per 1000'] = (df['Current Doctors'] / df['Population']) * 1000
df['Final Doctors per 1000'] = (df['Final Allocation'] / df['Population']) * 1000

# Create summary dataframe
summary_df = df[['Authority', 'Population', 'Current Doctors', 'Initial Allocation', 'Final Allocation', 'Current Doctors per 1000', 'Final Doctors per 1000']].copy()

# import ace_tools as tools; tools.display_dataframe_to_user(name="Doctor Allocation Summary with Accessibility", dataframe=summary_df)

summary_df


,Authority,Population,Current Doctors,Initial Allocation,Final Allocation,Current Doctors per 1000,Final Doctors per 1000
0,A1,16795,24,9,9,1.428997,0.535874
9,A10,61263,35,30,31,0.571307,0.506015
99,A100,3568,15,2,2,4.204036,0.560538
10,A11,17023,33,10,10,1.938554,0.587441
11,A12,42090,5,20,21,0.118793,0.498931
...,...,...,...,...,...,...,...
94,A95,39044,37,18,19,0.947649,0.486630
95,A96,52214,33,26,26,0.632014,0.497951
96,A97,62228,42,30,30,0.674937,0.482098
97,A98,49984,44,24,24,0.880282,0.480154


In [30]:
summary_df["Current Doctors per 1000"].var()

6.326196949968779

In [31]:
summary_df["Final Doctors per 1000"].var()

0.021226084500394905

### calculate fair allocation of doctors in a mock set up of 10 authorities - take into account the distances between authorities:
(use linear programming to solve)

In [32]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

# Generate sample data for 100 authorities
np.random.seed(42)
authorities = [f'A{i}' for i in range(1, 101)]
populations = np.random.randint(1000, 100000, size=100)
current_doctors = np.random.randint(1, 50, size=100)

data = {
    'Authority': authorities,
    'Population': populations,
    'Current Doctors': current_doctors
}

# Generate a random distance matrix with significant variations
distances = np.random.randint(1, 1000, size=(100, 100))
np.fill_diagonal(distances, 0)
distances = pd.DataFrame(distances, index=authorities, columns=authorities)

df = pd.DataFrame(data)

# Calculate total population and total number of doctors
total_population = df['Population'].sum()
total_doctors = df['Current Doctors'].sum()

# Calculate ideal number of doctors
df['Ideal Doctors'] = (df['Population'] / total_population) * total_doctors

# Adjust population based on accessibility
accessibility_factor = 0.1  # Example factor to weight the distance effect

def adjusted_population(row):
    authority = row['Authority']
    neighbors = distances.loc[authority]
    # Invert distances for accessibility (shorter distance = higher accessibility)
    accessibility = (1 / (neighbors + 1))  # +1 to avoid division by zero
    accessibility_score = (df.set_index('Authority').loc[neighbors.index, 'Population'] * accessibility).sum()
    return row['Population'] + accessibility_factor * accessibility_score

df['Adjusted Population'] = df.apply(adjusted_population, axis=1)
adjusted_total_population = df['Adjusted Population'].sum()

# Calculate adjusted ideal number of doctors
df['Adjusted Ideal Doctors'] = (df['Adjusted Population'] / adjusted_total_population) * total_doctors

# Linear Programming setup
c = np.ones(len(df))  # Coefficients for the objective function (minimize total discrepancy)

# Constraints
# A_eq and b_eq for the total number of doctors constraint
A_eq = np.ones((1, len(df)))
b_eq = [total_doctors]

# A_ub and b_ub for the integer bounds constraints
A_ub = np.vstack([np.eye(len(df)), -np.eye(len(df))])
b_ub = np.hstack([df['Adjusted Ideal Doctors'].apply(np.ceil), -df['Adjusted Ideal Doctors'].apply(np.floor)])

# Solve the linear program
res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, method='highs')

# Get the final allocation
df['Final Allocation'] = np.round(res.x).astype(int)

# Calculate doctors per 1000 people
df['Current Doctors per 1000'] = (df['Current Doctors'] / df['Population']) * 1000
df['Final Doctors per 1000'] = (df['Final Allocation'] / df['Population']) * 1000

# Create summary dataframe
summary_df = df[['Authority', 'Population', 'Current Doctors', 'Adjusted Ideal Doctors', 'Final Allocation', 'Current Doctors per 1000', 'Final Doctors per 1000']].copy()

# import ace_tools as tools; tools.display_dataframe_to_user(name="Doctor Allocation Summary with Accessibility", dataframe=summary_df)

summary_df


,Authority,Population,Current Doctors,Adjusted Ideal Doctors,Final Allocation,Current Doctors per 1000,Final Doctors per 1000
0,A1,16795,24,9.433955,9,1.428997,0.535874
1,A2,1860,1,3.450288,3,0.537634,1.612903
2,A3,77820,44,37.091092,37,0.565407,0.475456
3,A4,55886,8,27.074164,27,0.143149,0.483126
4,A5,7265,24,4.436749,4,3.303510,0.550585
...,...,...,...,...,...,...,...
95,A96,52214,33,26.444532,27,0.632014,0.517103
96,A97,62228,42,30.094445,31,0.674937,0.498168
97,A98,49984,44,24.196616,25,0.880282,0.500160
98,A99,41774,24,20.304867,21,0.574520,0.502705


In [33]:
summary_df["Current Doctors per 1000"].var()

6.3261969499687805

In [34]:
summary_df["Final Doctors per 1000"].var()

0.02158683615994569

### calculate fair allocation of doctors in a mock set up of 10 authorities - take into account the distances between authorities:
(enable definition for distance between authorities that is count as "nearby")

In [72]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

nearby_authority_threshold = 30 # number of KM to define nearby authority

# Generate sample data for 100 authorities
np.random.seed(42)
authorities = [f'A{i}' for i in range(1, 101)]
populations = np.random.randint(1000, 100000, size=100)
current_doctors = np.random.randint(1, 50, size=100)

data = {
    'Authority': authorities,
    'Population': populations,
    'Current Doctors': current_doctors
}

# Generate a random distance matrix with significant variations
distances = np.random.randint(1, 1000, size=(100, 100))
np.fill_diagonal(distances, 0)
distances = pd.DataFrame(distances, index=authorities, columns=authorities)

df = pd.DataFrame(data)

# Calculate total population and total number of doctors
total_population = df['Population'].sum()
total_doctors = df['Current Doctors'].sum()

# Calculate ideal number of doctors
df['Ideal Doctors'] = (df['Population'] / total_population) * total_doctors

# Adjust population based on accessibility
accessibility_factor = 0.1  # Example factor to weight the distance effect

def adjusted_population(row):
    authority = row['Authority']
    neighbors = distances.loc[authority]
    # Invert distances for accessibility (shorter distance = higher accessibility)
    accessibility = (1 / (neighbors + 1))  # +1 to avoid division by zero
    accessibility_score = (df.set_index('Authority').loc[neighbors.index, 'Population'] * accessibility).sum()
    return row['Population'] + accessibility_factor * accessibility_score

df['Adjusted Population'] = df.apply(adjusted_population, axis=1)
adjusted_total_population = df['Adjusted Population'].sum()

# Calculate adjusted ideal number of doctors
df['Adjusted Ideal Doctors'] = (df['Adjusted Population'] / adjusted_total_population) * total_doctors

# Linear Programming setup
c = np.ones(len(df))  # Coefficients for the objective function (minimize total discrepancy)

# Constraints
# A_eq and b_eq for the total number of doctors constraint
A_eq = np.ones((1, len(df)))
b_eq = [total_doctors]

# A_ub and b_ub for the integer bounds constraints
A_ub = np.vstack([np.eye(len(df)), -np.eye(len(df))])
b_ub = np.hstack([df['Adjusted Ideal Doctors'].apply(np.ceil), -df['Adjusted Ideal Doctors'].apply(np.floor)])

# Solve the linear program
res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, method='highs')

# Get the final allocation
df['Final Allocation'] = np.round(res.x).astype(int)

# Calculate doctors per 1000 people
df['Current Doctors per 1000'] = (df['Current Doctors'] / df['Population']) * 1000
df['Final Doctors per 1000'] = (df['Final Allocation'] / df['Population']) * 1000

# Calculate doctors within nearby_authority_thresholdkm
def doctors_within_nearby_authority_threshold_km(row, column):
    authority = row['Authority']
    nearby_authorities = distances.loc[authority][distances.loc[authority] <= nearby_authority_threshold].index
    total_doctors_nearby = df.set_index('Authority').loc[nearby_authorities, column].sum()
    return total_doctors_nearby

df[f'Current Doctors within {nearby_authority_threshold}km'] = df.apply(doctors_within_nearby_authority_threshold_km, axis=1, column='Current Doctors')
df[f'Final Doctors within {nearby_authority_threshold}km'] = df.apply(doctors_within_nearby_authority_threshold_km, axis=1, column='Final Allocation')
df[f'Current Doctors within {nearby_authority_threshold}km per 1000'] = (df[f'Current Doctors within {nearby_authority_threshold}km'] / df['Population']) * 1000
df[f'Final Doctors within {nearby_authority_threshold}km per 1000'] = (df[f'Final Doctors within {nearby_authority_threshold}km'] / df['Population']) * 1000

# Create summary dataframe
summary_df = df[['Authority', 'Population', 'Current Doctors', 'Adjusted Ideal Doctors', 'Final Allocation', 'Current Doctors per 1000', 'Final Doctors per 1000', f'Current Doctors within {nearby_authority_threshold}km', f'Final Doctors within {nearby_authority_threshold}km', f'Current Doctors within {nearby_authority_threshold}km per 1000', f'Final Doctors within {nearby_authority_threshold}km per 1000']].copy()

# import ace_tools as tools; tools.display_dataframe_to_user(name="Doctor Allocation Summary with Accessibility", dataframe=summary_df)

summary_df


,Authority,Population,Current Doctors,Adjusted Ideal Doctors,Final Allocation,Current Doctors per 1000,Final Doctors per 1000,Current Doctors within 30km,Final Doctors within 30km,Current Doctors within 30km per 1000,Final Doctors within 30km per 1000
0,A1,16795,24,9.433955,9,1.428997,0.535874,48,44,2.857993,2.619827
1,A2,1860,1,3.450288,3,0.537634,1.612903,94,161,50.537634,86.559140
2,A3,77820,44,37.091092,37,0.565407,0.475456,145,112,1.863274,1.439219
3,A4,55886,8,27.074164,27,0.143149,0.483126,16,59,0.286297,1.055721
4,A5,7265,24,4.436749,4,3.303510,0.550585,93,100,12.801101,13.764625
...,...,...,...,...,...,...,...,...,...,...,...
95,A96,52214,33,26.444532,27,0.632014,0.517103,230,212,4.404949,4.060214
96,A97,62228,42,30.094445,31,0.674937,0.498168,114,142,1.831973,2.281931
97,A98,49984,44,24.196616,25,0.880282,0.500160,119,92,2.380762,1.840589
98,A99,41774,24,20.304867,21,0.574520,0.502705,81,96,1.939005,2.298080


In [73]:
summary_df["Current Doctors per 1000"].var()

6.3261969499687805

In [74]:
summary_df["Final Doctors per 1000"].var()

0.02158683615994569

In [75]:
summary_df[f'Current Doctors within {nearby_authority_threshold}km per 1000'].var()

76.90920098968809

In [76]:
summary_df[f'Final Doctors within {nearby_authority_threshold}km per 1000'].var()

104.51928474879415